In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Rohlik Sales Forecasting Challenge

/content/drive/MyDrive/Colab Notebooks/Rohlik Sales Forecasting Challenge


## Data Cleaning

In [32]:
train_df = pd.read_csv('data/sales_train.csv')
test_df = pd.read_csv('data/sales_test.csv')
inv_df = pd.read_csv('data/inventory.csv')
calendar = pd.read_csv('data/calendar.csv')

### Date Formating


In [33]:
train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])
calendar['date'] = pd.to_datetime(calendar['date'])


### Merging Tables

In [34]:
test_inv_merge = pd.merge(test_df, inv_df, on = ['unique_id','warehouse'],how = 'left')
test_inv_cal_merge = pd.merge(test_inv_merge, calendar, on = ['date','warehouse'],how = 'left').sort_values(by='date')

In [35]:
train_inv_merge = pd.merge(train_df, inv_df, on = ['unique_id','warehouse'],how = 'left')
train_inv_cal_merge = pd.merge(train_inv_merge, calendar, on = ['date','warehouse'],how = 'left').sort_values(by='date')


In [36]:
test_inv_cal_merge.isnull().sum()

,0
unique_id,0
date,0
warehouse,0
total_orders,0
sell_price_main,0
type_0_discount,0
type_1_discount,0
type_2_discount,0
type_3_discount,0
type_4_discount,0


In [37]:
train_inv_cal_merge.isnull().sum()

,0
unique_id,0
date,0
warehouse,0
total_orders,52
sales,52
sell_price_main,0
availability,0
type_0_discount,0
type_1_discount,0
type_2_discount,0


In [38]:
# Drop na
train_inv_cal_merge.dropna(inplace=True)

In [11]:
# Drop features
drop_columns=["unique_id", "product_unique_id", "L2_category_name_en", "L3_category_name_en", "L4_category_name_en", "holiday_name", "winter_school_holidays", "school_holidays"]

train_inv_cal_merge = train_inv_cal_merge.drop(columns=drop_columns)
train_inv_cal_merge = train_inv_cal_merge.drop(columns=['availability'])

test_inv_cal_merge = test_inv_cal_merge.drop(columns=drop_columns)

In [12]:
# Create feature
train_inv_cal_merge['weekday'] = train_inv_cal_merge['date'].dt.weekday
train_inv_cal_merge['month'] = train_inv_cal_merge['date'].dt.strftime('%m')
train_inv_cal_merge['day'] = train_inv_cal_merge['date'].dt.strftime('%d')
train_inv_cal_merge['name_cat'] = train_inv_cal_merge['name'].str.split('_').str[0]

test_inv_cal_merge['weekday'] = test_inv_cal_merge['date'].dt.weekday
test_inv_cal_merge['month'] = test_inv_cal_merge['date'].dt.strftime('%m')
test_inv_cal_merge['day'] = test_inv_cal_merge['date'].dt.strftime('%d')
test_inv_cal_merge['name_cat'] = test_inv_cal_merge['name'].str.split('_').str[0]

train_inv_cal_merge = train_inv_cal_merge.drop(columns=['date'])
test_inv_cal_merge = test_inv_cal_merge.drop(columns=['date'])

In [13]:
# Categorise and encode data
data_type = {'warehouse':'category', 'L1_category_name_en':'category', 'month':'category', 'day':'category', 'name_cat':'category', 'name':'category'}
train_inv_cal_merge = train_inv_cal_merge.astype(data_type)
test_inv_cal_merge = test_inv_cal_merge.astype(data_type)

cat_col = ['warehouse', 'L1_category_name_en','month', 'day', 'name_cat', 'name']
for col in cat_col:
    train_inv_cal_merge[col] = train_inv_cal_merge[col].cat.codes
    test_inv_cal_merge[col] = test_inv_cal_merge[col].cat.codes

# train_inv_cal_merge['warehouse'] = train_inv_cal_merge['warehouse'].cat.codes
# train_inv_cal_merge['L1_category_name_en'] = train_inv_cal_merge['L1_category_name_en'].cat.codes
# train_inv_cal_merge['month'] = train_inv_cal_merge['month'].cat.codes
# train_inv_cal_merge['day'] = train_inv_cal_merge['day'].cat.codes
# train_inv_cal_merge['name_cat'] = train_inv_cal_merge['name_cat'].cat.codes
# train_inv_cal_merge['name'] = train_inv_cal_merge['name'].cat.codes

# test_inv_cal_merge['warehouse'] = test_inv_cal_merge['warehouse'].cat.codes
# test_inv_cal_merge['L1_category_name_en'] = test_inv_cal_merge['L1_category_name_en'].cat.codes
# test_inv_cal_merge['month'] = test_inv_cal_merge['month'].cat.codes
# test_inv_cal_merge['day'] = test_inv_cal_merge['day'].cat.codes
# test_inv_cal_merge['name_cat'] = test_inv_cal_merge['name_cat'].cat.codes
# test_inv_cal_merge['name'] = test_inv_cal_merge['name'].cat.codes

In [14]:
train_inv_cal_merge = train_inv_cal_merge.drop(columns=['name_cat'])
test_inv_cal_merge = test_inv_cal_merge.drop(columns=['name_cat'])

## Model

In [15]:
train_y = train_inv_cal_merge['sales']
train_X = train_inv_cal_merge.drop(columns='sales')
pred_X = test_inv_cal_merge

In [16]:
# Normalise
scaler = StandardScaler()

normalized_data_train_X = scaler.fit_transform(train_X)
train_X = pd.DataFrame(normalized_data_train_X, columns=train_X.columns)

normalized_data_pred_X = scaler.fit_transform(pred_X)
pred_X = pd.DataFrame(normalized_data_pred_X, columns=pred_X.columns)


In [17]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor

In [18]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, random_state=10)

### XGB Regressor

In [19]:
# param_grid = {
#     'n_estimators': [500],
#     'learning_rate': [0.2, 0.1],
#     'max_depth': [10],
#     'reg_alpha': [0],
#     'reg_lambda': [1],
#     'seed': [42],
#     'eval_metric': ['mae']
# }

In [20]:
# # Grid search
# xgb_reg = XGBRegressor(objective='reg:squarederror', random_state=42)

# clf = GridSearchCV(xgb_reg, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=3)
# clf.fit(X_train, y_train)

# print("Best parameters found: ", clf.best_params_)


In [21]:
# y_pred = clf.predict(X_test)
# rmse = mean_squared_error(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)

# print(f"RMSE: {rmse:.2f}")
# print(f"MAE: {mae:.2f}")

### XGB

In [22]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [23]:
params = {
    'objective': 'reg:squarederror',
    'n_estimators': 500,
    'learning_rate': 0.1,
    'max_depth': 15,
    'seed': 42,
    'eval_metric': 'mae',
}

In [24]:
evals=[(dtrain, 'train') , (dtest, 'test')]
bst = xgb.train(params, dtrain, num_boost_round=500, early_stopping_rounds=20, evals=evals)
y_pred = bst.predict(dtest)


rmse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:41:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mae:95.85429	test-mae:94.80515
[1]	train-mae:89.82656	test-mae:89.33536
[2]	train-mae:84.34506	test-mae:84.37021
[3]	train-mae:79.62053	test-mae:80.22320
[4]	train-mae:75.34334	test-mae:76.42677
[5]	train-mae:71.44020	test-mae:72.96500
[6]	train-mae:67.83198	test-mae:69.94354
[7]	train-mae:64.64036	test-mae:67.24265
[8]	train-mae:61.63870	test-mae:64.72695
[9]	train-mae:58.62024	test-mae:62.26784
[10]	train-mae:56.09561	test-mae:60.22573
[11]	train-mae:53.85182	test-mae:58.48897
[12]	train-mae:51.75375	test-mae:56.87567
[13]	train-mae:49.99540	test-mae:55.59706
[14]	train-mae:48.17483	test-mae:54.28379
[15]	train-mae:46.69104	test-mae:53.27393
[16]	train-mae:45.26549	test-mae:52.33598
[17]	train-mae:43.98924	test-mae:51.48471
[18]	train-mae:42.49536	test-mae:50.43613
[19]	train-mae:41.34617	test-mae:49.70774
[20]	train-mae:39.90779	test-mae:48.72156
[21]	train-mae:38.73828	test-mae:47.98502
[22]	train-mae:38.05159	test-mae:47.64940
[23]	train-mae:37.23701	test-mae:47.21226
[2

In [25]:
feature_gain = bst.get_score(importance_type='weight')
sorted(feature_gain.items(), key=lambda x: x[1], reverse=True)

[('total_orders', 315037.0),
 ('sell_price_main', 308893.0),
 ('name', 252309.0),
 ('warehouse', 84865.0),
 ('weekday', 81127.0),
 ('day', 63359.0),
 ('month', 61034.0),
 ('type_6_discount', 32406.0),
 ('type_0_discount', 28619.0),
 ('shops_closed', 19796.0),
 ('L1_category_name_en', 15083.0),
 ('type_4_discount', 10895.0),
 ('holiday', 9145.0),
 ('type_2_discount', 4162.0),
 ('type_5_discount', 2296.0),
 ('type_1_discount', 482.0),
 ('type_3_discount', 68.0)]